In [1]:
from main_model import MainModel
import torch
from config import T, C, N_MFCC, H, W, NUM_CLASSES

T = 10
H = 512
W = 512

model = MainModel(
    T=T, C=C, H=H, W=W,
    N_MFCC=N_MFCC,
    num_classes=NUM_CLASSES,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on device: {device}")
model = model.to(device)

# Count the number of parameters in the model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

d:\Program Files\Python312\Lib\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Running on device: cuda


16327503

In [2]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=10, gamma=0.1)

In [3]:
audio = torch.randn(1, 4*T, 13)
visual = torch.randn(1, T, C, H, W)
targets = torch.randint(-1, NUM_CLASSES, (1, T, 25))
boxes = torch.rand(1, T, 25, 4)

In [8]:
audio_features = model.forward_audio_encoder(audio.to(device))
visual_features = model.forward_visual_encoder(visual.to(device))
fused_features = model.forward_fusion(visual_features, audio_features)
head_output = model.forward_head(fused_features)

In [5]:
model.eval()
with torch.no_grad():
    head_output = model(audio.to(device), visual.to(device))

In [6]:
head_output.shape

torch.Size([10, 25, 6])

## Contrastive Loss

In [ ]:
import torch.nn as nn

class ContrastiveLoss(nn.Module):
    def __init__(self, fmap_size, in_channels=128, out_dim=128, tau=1):
        super(ContrastiveLoss, self).__init__()
        self.temperature = tau
        self.in_channels = in_channels
        self.fmap_size = fmap_size
        self.in_channels = in_channels
        self.conv = nn.Conv2d(
            in_channels=in_channels,
            out_channels=1,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.fc = nn.Linear(fmap_size, out_dim)
        self.tau = tau

    def forward(self, audio_features, visual_features, active_frames):
        """
        Args:
            audio_features: (B, T, C)
            visual_features: (B, T, C, hi, wi)
            active_frames: (B, T)
        """
        batch_size, T, ... = audio_features.size()
        V = V[active_frames]
        V = visual_features.view(batch_size*T, -1)
        
        visual_features = self.conv(visual_features)
        visual_features = self.fc(visual_features.view(visual_features.size(0), -1))
        
        A = audio_features.view(audio_features.size(0), -1)
        V = visual_features.view(visual_features.size(0), -1)
        A = A[active_frames]
        A = A / torch.norm(A, dim=1, keepdim=True)
        V = V / torch.norm(V, dim=1, keepdim=True)
        s = torch.matmul(
            A.view(A.size(0), -1).T,
            V.view(V.size(0), -1)
        )
        exp_s = torch.exp(s / self.tau)
        exp_diag = exp_s.diagonal(offset=0, dim1=-2, dim2=-1)
        sum_exp = exp_s.sum(dim=1, keepdim=True) - exp_diag
        # Compute the contrastive loss
        return -torch.log(exp_diag / sum_exp).mean()


In [ ]:

loss = ContrastiveLoss()

# Test epoch

In [1]:
from dataset import AVADataset, AVADataLoader
from train import Trainer
import torch
from main_model import MainModel

KeyboardInterrupt: 

In [ ]:
from config import T, C, N_MFCC, H, W
T = 5

train_dataset = AVADataset(
    mode="train",
    N_MFCC=N_MFCC,
    T=T,
    C=C,
    H=H,
    W=W,
) 
train_loader = AVADataLoader(
    train_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=0,
)

In [ ]:
model = MainModel(
    T=T,
    C=C,
    H=H,
    W=W,
    N_MFCC=N_MFCC,
    num_classes=NUM_CLASSES,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on device: {device}")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=10, gamma=0.1
)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
)

d:\Program Files\Python312\Lib\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Running on device: cuda


In [7]:
trainer.do_one_epoch(
    train_loader
)

Training:   0%|          | 0/3576 [00:00<?, ?it/s]

Training:  22%|██▏       | 801/3576 [07:45<26:12,  1.77it/s, av=195, det=5.12e+4, loss=1.02e+5]  

KeyboardInterrupt: 